# 🔬 ViTHSD - Dataset A - FlanT5 (Fine-tuning)

**Model 2/3 cho Dataset A**

- **Model**: google/flan-t5-base
- **Dataset**: A (Original ViTHSD - chỉ content + labels)
- **Task**: Multi-Label Classification (11 labels)
- **Technique**: Full Fine-tuning

### 📌 Label Constraint
- Each category can only have **ONE level**: offensive OR hate (not both)

## Step 1: Setup Environment

In [1]:
# Cài đặt các thư viện cần thiết
!pip uninstall -y protobuf
!pip install -q protobuf==3.20.0
!pip install -q transformers datasets accelerate sentencepiece
!pip install -q scikit-learn pandas numpy tqdm openpyxl

# Suppress warnings
import os
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['GRPC_VERBOSITY'] = 'ERROR'
warnings.filterwarnings('ignore')

print("✅ Dependencies installed!")

Found existing installation: protobuf 6.33.0
Uninstalling protobuf-6.33.0:
  Successfully uninstalled protobuf-6.33.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.0 which is incompatible.
google-cloud-secret-manager 2.25.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
google-cloud-vision 3.11.0 requires protobuf!=4.21.0,!=4.

In [2]:
# 🎮 GPU Auto-Detection & Optimization
import torch
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"\n{'='*70}")
    print(f"🎮 DETECTED: {num_gpus} GPU(s)")
    print(f"{'='*70}")
    
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
    
    # Auto-configure based on number of GPUs
    if num_gpus == 1:
        # Single GPU (P100)
        print(f"\n{'='*70}")
        print("⚙️  SINGLE GPU MODE (P100 Optimized)")
        print(f"{'='*70}")
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
        
        BATCH_SIZE_FLANT5 = 8  # P100 (16GB) - FlanT5 nhẹ hơn Qwen
        
        GPU_CONFIG = {
            'num_gpus': 1,
            'device': 'cuda:0',
            'batch_sizes': {
                'flant5': BATCH_SIZE_FLANT5
            },
            'gradient_accumulation_steps': 1
        }
        
    elif num_gpus >= 2:
        # Multi-GPU (T4 x2)
        print(f"\n{'='*70}")
        print("⚡ MULTI-GPU MODE (T4 x2 Maximum Performance)")
        print(f"{'='*70}")
        
        BATCH_SIZE_FLANT5 = 16  # T4 x2 (32GB)
        
        GPU_CONFIG = {
            'num_gpus': num_gpus,
            'device': 'cuda',
            'batch_sizes': {
                'flant5': BATCH_SIZE_FLANT5
            },
            'gradient_accumulation_steps': 1
        }
    
    print(f"\n📊 Configured Batch Size:")
    print(f"   FlanT5: {GPU_CONFIG['batch_sizes']['flant5']}")
    print(f"\n{'='*70}\n")
    
else:
    print("❌ No GPU detected! This notebook requires GPU.")
    GPU_CONFIG = None

PyTorch version: 2.6.0+cu124
CUDA available: True

🎮 DETECTED: 2 GPU(s)
GPU 0: Tesla T4
GPU 1: Tesla T4

⚡ MULTI-GPU MODE (T4 x2 Maximum Performance)

📊 Configured Batch Size:
   FlanT5: 16




## Step 2: Import và Setup Paths

In [3]:
import sys
import os

# Copy files ra working directory
!cp -r /kaggle/input/vithsd-experiment/* /kaggle/working/

os.chdir("/kaggle/working")
sys.path.insert(0, "/kaggle/working")

print("📁 Files copied to working directory")

📁 Files copied to working directory


In [4]:
# Import modules
from config import FINAL_LABELS, DATA_DIR, OUTPUT_DIR
from data_preparation import load_dataset_A
from evaluation import Evaluator
from models import create_model

print("✅ All modules imported!")
print(f"📊 Labels: {len(FINAL_LABELS)}")

✅ All modules imported!
📊 Labels: 11


## Step 3: Load Dataset A

In [5]:
# Load Dataset A
train_texts, train_labels = load_dataset_A('train')
test_texts, test_labels = load_dataset_A('test')

print(f"📊 Dataset A:")
print(f"   Train: {len(train_texts)} samples")
print(f"   Test: {len(test_texts)} samples")

[ViHSDLoader] Loaded 7000 samples from train
[ViHSDLoader] Loaded 1800 samples from test
📊 Dataset A:
   Train: 7000 samples
   Test: 1800 samples


## Step 4: Train FlanT5

In [6]:
# Initialize
evaluator = Evaluator()

# FlanT5 - Dataset A
print("="*70)
print("🚀 FLANT5 - DATASET A")
print("="*70)

model_flant5 = create_model(
    "flant5", 
    dataset_type="A",
    batch_size=GPU_CONFIG['batch_sizes']['flant5'],  # Auto-adjusted
    num_epochs=3,
    learning_rate=3e-5
)

# Train
model_flant5.train(train_texts, train_labels)

# Predict
pred_flant5, _ = model_flant5.predict(test_texts)

# Evaluate
result = evaluator.evaluate(test_labels, pred_flant5, "FlanT5", "A")
evaluator.print_result(result)

🚀 FLANT5 - DATASET A


E0000 00:00:1765782032.676299      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765782032.789141      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[FlanT5_MultiLabel] Training...
  Device: cuda
  Original samples: 7000
  Mode: Standard (Dataset A)
  [Oversampling] Added 540 samples for minority labels


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

  Epoch 2: avg_loss = 0.1026


Epoch 3/3: 100%|██████████| 472/472 [08:36<00:00,  1.09s/it, loss=0.1063]


  Epoch 3: avg_loss = 0.0980
  ✓ Training completed


Predicting: 100%|██████████| 113/113 [00:56<00:00,  2.01it/s]


📊 FlanT5 on Dataset A (Multi-Label)
  Subset Accuracy (Exact Match): 0.4622
  Hamming Loss:                  0.1174
  F1 Samples:                    0.5074
  F1 Macro:                      0.1311
  F1 Micro:                      0.4684
  Precision (Samples):           0.5033
  Recall (Samples):              0.5210

  F1 per Label:
    normal                   : 0.6732
    individuals#hate         : 0.3583
    groups#hate              : 0.3493
    race#hate                : 0.0312
    politics#hate            : 0.0299


## Step 5: Save Results

In [7]:
from pathlib import Path
import json

# Create output directory
output_dir = Path("/kaggle/working/outputs_datasetA_flant5")
output_dir.mkdir(exist_ok=True)

# Save results
evaluator.save_results(output_dir)

# Save report
report = evaluator.generate_report()
with open(output_dir / "experiment_report_flant5.txt", 'w') as f:
    f.write(report)

print(f"\n✅ Results saved to {output_dir}")

✓ Saved results to /kaggle/working/outputs_datasetA_flant5/experiment_results.json
✓ Saved comparison matrix to /kaggle/working/outputs_datasetA_flant5/comparison_matrix.csv
✓ Saved full comparison to /kaggle/working/outputs_datasetA_flant5/full_comparison.csv

✅ Results saved to /kaggle/working/outputs_datasetA_flant5


In [8]:
# 💾 Lưu model đã train
import torch
from pathlib import Path

models_dir = Path("/kaggle/working/outputs_datasetA_flant5/models")
models_dir.mkdir(exist_ok=True)

# Lưu FlanT5 weights
try:
    torch.save(model_flant5.model.state_dict(), models_dir / "flant5_model.pt")
    print("✅ Saved: flant5_model.pt")
except Exception as e:
    print(f"⚠️ Không thể lưu FlanT5 model: {e}")

print(f"\n📁 Models saved to: {models_dir}")

✅ Saved: flant5_model.pt

📁 Models saved to: /kaggle/working/outputs_datasetA_flant5/models


In [9]:
# 📦 Tạo file ZIP để tải về
import shutil
from pathlib import Path

zip_path = "/kaggle/working/vithsd_datasetA_flant5_results"
shutil.make_archive(zip_path, 'zip', "/kaggle/working/outputs_datasetA_flant5")

print(f"✅ Created: {zip_path}.zip")
print(f"📦 Zip size: {Path(zip_path + '.zip').stat().st_size / (1024*1024):.2f} MB")

✅ Created: /kaggle/working/vithsd_datasetA_flant5_results.zip
📦 Zip size: 876.42 MB


## Step 6: Per-Label Analysis

In [10]:
# Per-label F1 Analysis
print("\n" + "="*100)
print("📈 DATASET A - PER-LABEL F1 SCORES (FlanT5)")
print("="*100)

for r in evaluator.results:
    print(f"\n{r['model']}:")
    print("-" * 80)
    f1_per_label = r['f1_per_label']
    sorted_labels = sorted(f1_per_label.items(), key=lambda x: x[1], reverse=True)
    for label, f1 in sorted_labels:
        bar_length = int(f1 * 40)
        bar = "█" * bar_length + "░" * (40 - bar_length)
        print(f"  {label:25s} | {bar} | {f1:.4f}")


📈 DATASET A - PER-LABEL F1 SCORES (FlanT5)

FlanT5:
--------------------------------------------------------------------------------
  normal                    | ██████████████████████████░░░░░░░░░░░░░░ | 0.6732
  individuals#hate          | ██████████████░░░░░░░░░░░░░░░░░░░░░░░░░░ | 0.3583
  groups#hate               | █████████████░░░░░░░░░░░░░░░░░░░░░░░░░░░ | 0.3493
  race#hate                 | █░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ | 0.0312
  politics#hate             | █░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ | 0.0299
  individuals#offensive     | ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ | 0.0000
  groups#offensive          | ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ | 0.0000
  religion#offensive        | ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ | 0.0000
  religion#hate             | ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ | 0.0000
  race#offensive            | ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ | 0.0000
  politics#offensive        | ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░

## 🎉 Done - FlanT5 on Dataset A!

**Model đã chạy trên Dataset A (Original):**
- ✅ FlanT5 (google/flan-t5-base)

Kết quả: `/kaggle/working/outputs_datasetA_flant5/`